In [ ]:
%matplotlib notebook
import numpy as np
import scipy.constants as c

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.patheffects as PathEffects
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

from magpie_data import DMFromArray, NeLMap, Interferogram

# Interferometry

#### 1. How to use this template

This template is both a tutorial and a repository of useful code blocks. In it you will find out how to:
* Load and plot raw interferograms
* Assign a scale, origin and region of interest
* Abel invert using a simple Gaussian fit
* Solve simultaneous equations for finding neutral and electron densities
* Take lineouts (it's kinda hidden but it's there!)
* Create images with multiple plots

#### 2. Filenames
This uses the example data from plasma gun shot g23, which includes interferometry at 532 nm and 355 nm. You will need to change the first line to point to where you have stored the interferometry data.

I recommend opening a pre-shot shadow image in Photoshop for both wavelengths. 

The 355 nm has a left/right flip compared to the 532 nm, due to a beam splitter. Flip the shadow image in photoshop.

Secondly, determine the rotation. 
Here you can use a straight edge in your hardware and the photoshop ruler tool to determine the rotation angle that rotates the image onto your desired coordinate system. Here the rotations are small, a few degrees due to the mirrors in the optical path slightly rotating the image. Rotate the image in photoshop as well.

You can also determine the scale in Photoshop - pick two points which you know the separation between in mm, and measure the separation in pixels. Then set the scale as pixels per mm. Here we use the diameter of the plasm gun, 6.25 mm. Both 532 nm and 355 nm use similar cameras and optics, so the scale is similar.

In [ ]:
%cd "~\Google Drive\MAGPIE\data\2018\s0823_18 Plasma Gun TS\532 nm interferometry"

rot=-2
scale=130

##Load data

g23_532=NeLMap('g23 532 nm fringe shift2.csv', scale=scale, rot_angle=rot, multiply_by=np.pi, flip_lr=False)
g23_532_if=Interferogram('g23 shot.JPG', scale=scale, rot_angle=rot)
g23_532_bkif=Interferogram('g20 bk.JPG', scale=scale, rot_angle=rot)

In [ ]:
%cd "~\Google Drive\MAGPIE\data\2018\s0823_18 Plasma Gun TS\355 nm interferometry"

rot=-3.25
scale=130

##Load data

g23_355=NeLMap('g23 355 nm fringe shift3.csv', scale=scale, rot_angle=rot, multiply_by=np.pi, flip_lr=True)
g23_355_if=Interferogram('g23 shot.JPG', scale=scale, rot_angle=rot, flip_lr=True)
g23_355_bkif=Interferogram('g20 bk.JPG', scale=scale, rot_angle=rot,flip_lr=True)

#### 3. Set origin and extent

IMPORTANT NOTE: To ensure consistency with matplotlib and arrays, we use (vertical, horizontal) as the coordinate system, where the pixel origin is in the top left of the image. This is in contrast the usual physics convention of (horizontal, vertical) and an origin in the bottom right. Fortunately, Photoshop uses the same origin convention, which helps a lot.

Using photoshop on your (flipped and) rotated image, choose the origin of your image in pixel coordinates. This will be the origin (0,0) of the image in physical coordinates (eg mm). You may need to use the ruler tool and several points to determine this - for the plasma gun, I simply take a line along the top of the gun (which should now be horizontal, you did rotate the image!) and find the midpoint.

Then determine how much of the original image you want to keep, in mm. This can take a bit of trial and error, as if you select a region which is out of bounds, the code currently does some odd things. Use verbose  = True to check the pixels you are asking for are inside the array.

In [ ]:
## Set origin and crop data
## Origin determined using photoshop.

o_532=(2555,2560)
e=[0,6,-7,7]
g23_532.set_origin(origin=o_532, extent=e)
g23_532_if.set_origin(origin=o_532, extent=e)
g23_532_bkif.set_origin(origin=o_532, extent=e)

## Set origin and crop data
## Origin determined using photoshop.

o_355=(2516,1824)

g23_355.set_origin(origin=o_355, extent=e)
g23_355_if.set_origin(origin=o_355, extent=e)
g23_355_bkif.set_origin(origin=o_355, extent=e)

#### 4. Plot Data

This code is a bit involved, because it produces a "production ready" figure, Fig 2 in Hare et al. PPCF (2019)

In [ ]:
## We use the gridspec interface to get the figures exactly where we want them

grid = gridspec.GridSpec(2,3, width_ratios=[1,1,1], height_ratios=[1,1])
fig = plt.figure(figsize=(17.0/2.54,2.2))
fig.subplots_adjust(wspace=0.05, hspace=0.1,left=0.06, bottom=0.19, right=0.9, top=0.95)

## these axis objects are what we use to plot
ax0 = fig.add_subplot(grid[0])
ax1 = fig.add_subplot(grid[1])
ax2 = fig.add_subplot(grid[2])

ax3 = fig.add_subplot(grid[3])
ax4 = fig.add_subplot(grid[4])
ax5 = fig.add_subplot(grid[5])

## This section turns off the various unwanted tick labels and set the axis labels.

for a in [ax0,ax1,ax2,ax3,ax4,ax5]:
    a.tick_params(labelsize=10, pad=5, length=2, width=1)
    a.set_ylim([0,6])
    a.set_xticks([-5,0,5])
        
for a in [ax0,ax3]:
    a.set_ylabel('z (mm)',fontsize=10)
    
for a in [ax1,ax2,ax4,ax5]:
    a.set_yticks([])

for a in [ax0,ax1,ax2]:
    a.tick_params(labelbottom='off')
    
for a in [ax3,ax4,ax5]:
    a.set_xlabel('x (mm)',fontsize=10, labelpad=-2)
    
## Here we plot the actual data: RGB images for the inteferograms, a colour map for the phase.
    
clim=[0,10]    

g23_532_bkif.plot_data_mm(ax=ax0)
g23_532_if.plot_data_mm(ax=ax1)
im=g23_532.plot_data_mm(ax=ax2, clim=clim)

g23_355_bkif.plot_data_mm(ax=ax3)
g23_355_if.plot_data_mm(ax=ax4)
g23_355.plot_data_mm(ax=ax5, clim=clim)
    
## Positioning of colour bar

[[x00,y00],[x01,y01]] = ax2.get_position().get_points()
[[x10,y10],[x11,y11]] = ax5.get_position().get_points()
pad = 0.01; width = 0.02
cbar_ax = fig.add_axes([x11+pad, y10, width, (y01-y10)])
axcb = fig.colorbar(im, cax=cbar_ax,ticks=[0,5, 10])
cbar_ax.set_ylabel(r'Phase (rad)',fontsize=10, labelpad=4)

## Labels, with some funky PathEffects to produce a black outline to white text. Nice, eh?

txt=ax0.text(x=0.01, y=0.99, va='top',ha='left',s=r'a)', color='white',fontsize=10,transform=ax0.transAxes);
txt.set_path_effects([PathEffects.withStroke(linewidth=1, foreground='k')])
txt=ax1.text(x=0.01, y=0.99, va='top',ha='left',s=r'b)', color='white',fontsize=10,transform=ax1.transAxes);
txt.set_path_effects([PathEffects.withStroke(linewidth=1, foreground='k')])

ax2.text(x=0.01, y=0.99, va='top',ha='left',s=r'c)', color='white',fontsize=10,transform=ax2.transAxes);
ax3.text(x=0.01, y=0.99, va='top',ha='left',s=r'd)', color='white',fontsize=10,transform=ax3.transAxes);
ax4.text(x=0.01, y=0.99, va='top',ha='left',s=r'e)', color='white',fontsize=10,transform=ax4.transAxes);
ax5.text(x=0.01, y=0.99, va='top',ha='left',s=r'f)', color='white',fontsize=10,transform=ax5.transAxes);

#### 5. Abel inversion

This uses Jakub's fast and simple Abel inversion: row by row, we fit a Gaussian (centered at x = 0 mm) and calculate the analytica Abel transform (which means the result is smooth). 

The call to abel_invert returns two DataMaps: one is the Gaussian fit to the line integrated phase (units: radians), the second is the abel inverted phase (units: radians/cm). Both are stored as properties of the original object, so we don't actually need them as a separate objects.

In [ ]:
y_lim = [0, 6]
x_range = 7
g23_532_fit,g23_532_abel=g23_532.abel_invert(y_lim, x_range, parameters={'sigma': {'min':0.001,'max':7}})
g23_355_fit,g23_355_abel=g23_355.abel_invert(y_lim, x_range)

#### 6. Separate neutral and electron density contributions to the phase

Here we solve the simultaneous equations, p156 of *Muraoka, Katsunori, and Mitsuo Maeda. Laser-Aided Diagnostics of Plasmas and Gases. Series in Plasma Physics. Bristol: Institute of Physics Pub, 2001.*

We use the Abel transformed phase maps for the 532 nm and 355 nm interferometry. I am uncertain whether we get the same result if we 1) solve the simultaneous equations and then Abel invert or 2) Abel invert and then solve the simulatneous equations. I think we sould - the Abel inversion is linear - but I am not sure.

In [ ]:
r_e=2.82e-13 # cm, classical electron radius
alpha=4*np.pi**2*11*0.148e-24 # cm3 polarisation http://ctcp.massey.ac.nz/Tablepol2014.pdf note 4pi^2 from Sergey notes

# Cropped data from Abel Inversion
phi_g=g23_532.abel.data_c
phi_b=g23_355.abel.data_c
l_g=532e-7 # cm, laser wavelength for 532 nm, green
l_b=355e-7 # cm, laser wavelength for 355 nm, blue

# Solve simultaneous equations for electron density n_e and neutral density n_a
n_e_data=-1/r_e*(phi_g*l_g-phi_b*l_b)/(l_b**2-l_g**2)
n_a_data=-1/alpha*(phi_g/l_g-phi_b/l_b)/(l_g**-2-l_b**-2)

# Create new DataMap objects to hold the results.
extent=[-x_range,x_range,y_lim[0],y_lim[1]]
origin=[int(len(n_e_data)+y_lim[0]*130), int(len(n_e_data[0])/2)]
n_e = DMFromArray(n_e_data, scale=130, extent=extent, origin=origin)
n_a = DMFromArray(n_a_data, scale=130, extent=extent, origin=origin)

#### 7. Plot electron and neutral densities

In [ ]:
## Use gridspec to line everything up

grid = gridspec.GridSpec(1,4, width_ratios=[1,0.1,1,1])
fig = plt.figure(figsize=(6.69,1.6))
fig.subplots_adjust(wspace=0.15, hspace=0.0,left=0.06, bottom=0.25, right=0.97, top=0.86)

ax0 = fig.add_subplot(grid[0])
ax1 = fig.add_subplot(grid[2])
ax2 = fig.add_subplot(grid[3])

## Create a lineout to compare the Gaussian fit to the real data.

zz=5
xx=7
g=g23_532
g.create_lineout(start=(zz,-xx), end=(zz,xx), lineout_width_mm=1)
g.fit.create_lineout(start=(zz,-xx), end=(zz,xx), lineout_width_mm=1)

## Plot lineout

ax0.plot(g.mm, g.lo, label='532 nm data', c='g')
ax0.plot(g.fit.mm, g.fit.lo, label='532 nm fit', ls=':', c='g')
ax0.set_ylim([2,4.5])
ax0.set_yticks([2,3,4])
ax0.set_xlim([-7,7])
ax0.set_ylabel('Phase (rad)',fontsize=10)

### Nice grid lines (important)

ax0.xaxis.set_minor_locator(MultipleLocator(2.5))
ax0.yaxis.set_minor_locator(MultipleLocator(1))
ax0.grid(which='minor', color='gray',linestyle=':')
ax0.grid()

## Now plot the electron and neutral densities.

ne_im = n_e.plot_data_mm(ax=ax1, clim=[0,10], multiply_by=1e-17)
na_im = n_a.plot_data_mm(ax=ax2, clim=[-22,22], multiply_by=1e-17)
na_im.set_cmap('coolwarm')
        
### Colourbar fun times. These are a pain!
ax1_divider = make_axes_locatable(ax1)
cax1 = ax1_divider.append_axes("top", size="7%", pad="10%")
cb1 = plt.colorbar(ne_im, cax=cax1, orientation="horizontal", ticks=[0,5,10])
cax1.xaxis.set_ticks_position("top")
cax1.set_xlabel(r'$n_e$ ($\times$ 10$^{17}$ cm$^{-3}$)',fontsize=10, labelpad=-8)
cax1.xaxis.set_label_position('top') 
cax1.set_xticklabels(['0', '','10'])  # horizontal colorbar
cax1.tick_params(labelsize=9, pad=2, length=3, width=1, direction='in')


ax2_divider = make_axes_locatable(ax2)
cax2 = ax2_divider.append_axes("top", size="7%", pad="10%")
cb2 = plt.colorbar(na_im, cax=cax2, orientation="horizontal", ticks=[-20,0,20])
cax2.xaxis.set_ticks_position("top")
cax2.set_xlabel(r'$n_a$ ($\times$ 10$^{17}$ cm$^{-3}$)',fontsize=10, labelpad=-8)
cax2.xaxis.set_label_position('top') 
cax2.set_xticklabels(['-20', '','20'])  # horizontal colorbar
cax2.tick_params(labelsize=9, pad=2, length=3, width=1, direction='in')


### Set axis labels and limits
for a in [ax0,ax1,ax2]:
    a.set_xlabel('r (mm)',fontsize=10, labelpad=-1)
    a.tick_params(labelsize=9, pad=2, length=4, width=1)
    
ax1.set_ylabel('z (mm)',fontsize=10)
ax2.tick_params(labelleft=False)
ax0.set_xlabel('x (mm)',fontsize=10, labelpad=-1)

### Labels
ax0.text(x=0.01, y=0.99, va='top',ha='left',s=r'a)', color='black',fontsize=9,transform=ax0.transAxes);
ax1.text(x=0.01, y=0.99, va='top',ha='left',s=r'b)', color='white',fontsize=9,transform=ax1.transAxes);
ax2.text(x=0.01, y=0.99, va='top',ha='left',s=r'c)', color='black',fontsize=9,transform=ax2.transAxes);

#### 8. Find density along Thomson Scattering chord

In [ ]:
zz=5 # location of horizontal TS chord (5 mm for plasma gun)
rr=7 # lineout between +/- rr
n_e.create_lineout(start=(zz,-rr), end=(zz,rr), lineout_width_mm=0.2) # width of 200 um in z, roughly right for plasma gun.
fig, ax = plt.subplots(figsize=(5,3))

ax.plot(n_e.mm, n_e.lo/1e17, lw=3)
ax.set_ylim([0,3])
ax.set_yticks(range(4))
ax.set_xlim([-7,7])
ax.set_ylabel(r'$n_e$ ($\times$ 10$^{17}$ cm$^{-3}$)',fontsize=10)
ax.set_xlabel('r (mm)',fontsize=10, labelpad=-1)

ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid(which='minor', color='gray',linestyle=':')
ax.grid()

x4=0.4
E_space=1.13
x_E_fibres=np.linspace(-3*E_space, 3*E_space, 7)+x4
F_space=0.43
x_F_fibres=np.linspace(-3*F_space, 3*F_space, 7)+x4

for e in x_E_fibres:
    ax.plot([e,e], [0,3], c='#3DC1C9', ls=':', lw=2)
    
for f in x_F_fibres:
    ax.plot([f,f], [0,3], c='#EE2331', ls=':', lw=2)